# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [6]:
grouped = data.groupby('race').call
grouped.value_counts()

race  call
b     0.0     2278
      1.0      157
w     0.0     2200
      1.0      235
Name: call, dtype: int64

In [25]:
#1. What test is appropriate for this problem? Does CLT apply?
# The appropriate test for this problem is a chi-squared test, since we are dealing with categorical variables
# and trying to find out if the two categorical variables are independent. That is, we are trying to see if 
# how a names sounds (black or white) has a significant effect on getting a call. To check for the CLT, we must make sure
# that n*p & n*(1-p) > 10.  Both of these conditions are satisfied as we can see from the dataframe below.

total_black_calls = len(data[data.race=='b'].call)
total_white_calls = len(data[data.race=='w'].call)
black_success = sum(data[data.race=='b'].call)
white_success = sum(data[data.race=='w'].call)
black_failure = total_black_calls - black_success
white_failure = total_white_calls - white_success

prop_b = black_success/total_black_calls
prop_w = white_success/total_white_calls

pd.DataFrame({'black':{'all people':total_black_calls, 'callback':black_success, 'success rate':prop_b, 'no call':black_failure},
            'white':{'all people':total_white_calls, 'callback':white_success, 'success rate':prop_w, 'no call':white_failure}})

,black,white
all people,2435.000000,2435.000000
callback,157.000000,235.000000
no call,2278.000000,2200.000000
success rate,0.064476,0.096509


2. What are the null and alternate hypotheses?
The null hypothesis is that there is no difference between the rate of calls between black and white sounding names. The alternative hypothesis is that there is a difference between the rate of calls b/t black and white sounding names. Another way of stating the null is that the success rate of blacks is equal to the success rate of whites success rate of blacks - success rate of whites = 0. Another way of stating the alternative is that the success rate of blacks is not equal to the succes rate of whites success rate of blacks - success rate of whites ≠ 0

In [32]:
#3. Compute margin of error, confidence interval, and p-value.
# The chi-square test tests the difference between observed and theoretical data
# The theoretical data suggests that there is no difference in success rate i.e both black and white should be evenly
# distributed

total_success = black_success + white_success
total_failure = black_failure + white_failure
even_success = total_success/2
even_failure = total_failure/2

observed = [black_success, black_failure, white_success, white_failure]
expected = [even_success, even_failure, even_success, even_failure]


In [37]:
#Calculate the chi-squared test using 1 degree of freedom (number of cells - 1)

xstat, pvalue = stats.chisquare(f_obs = observed, f_exp = expected, ddof=2)
print('Chi Square Stat:', xstat)
print('Pvalue:', pvalue)

Chi Square Stat: 16.8790504143
Pvalue: 3.98388683759e-05


In [44]:
#Calculate the margin of error - we will use the standard error & 95% confidence interval. Z score = 1.96

zscore95 = 1.96
margin_error = zscore95 * np.sqrt((total_success/len(data))*(1-(total_success/len(data)))*
                                  (1/total_black_calls + 1/total_white_calls))
print('Margin of Error:', margin_error)


Margin of Error: 0.0152819123109


In [50]:
#Calculate the 95% confidence interval

CI_95 = [prop_w - prop_b - margin_error, 
         prop_w - prop_b + margin_error]
print('95% Confidence Interval:', CI_95)

95% Confidence Interval: [0.016750941898551489, 0.047314766520339682]


4. Write a story describing the statistical significance in the context or the original problem

Since the p value(3.98388683759e-05) is significantly below the threshold of 0.05, we reject the null hypothesis nd accept the alternative that there is a difference between the rate of calls between black and white sounding names. In this case, white sounding names are more likely to get calls. Furthermore, we are 95% confident that the true difference in callback succes lies somehwere between 0.01675 and 0.04731.According to the data, white sounding names are more likely to get calls than black sounding names.

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? 

If not, how would you amend your analysis? It is unclear whether or not race is the number one predictor of callback success. We have only looked at race compared to see if it had a signifcant effect on callback succes, however, it is possible that other variables also have a significant impact as well. In order to test which variables are most impactful, we could run a regression analysis to see which coefficients are largest i.e. which have the biggest prediction power.